# 데이터 불러오기

In [1]:
import pandas as pd
from google.colab import drive 
drive.mount('/content/mnt')


Drive already mounted at /content/mnt; to attempt to forcibly remount, call drive.mount("/content/mnt", force_remount=True).


In [2]:
test_df = pd.read_csv("/content/mnt/MyDrive/contest/dacon/fake_news/1.Data/news_test.csv")

# 시간 측정 시작

In [3]:
import time
start = time.time()

# Library 불러오기

In [4]:
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done
Reading package lists... Done
B

In [5]:
from tensorflow import debugging
from tensorflow.keras.models import load_model
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Mecab
import re
import os
import math
import gensim
import pickle



debugging.set_log_device_placement(True)

# 각종 모델 불러오기

In [6]:
# pretrained_model
word2vec = gensim.models.Word2Vec.load('/content/mnt/MyDrive/contest/dacon/fake_news/4.pre_trained_embedding/ko.bin')
# tokenizer
with open('/content/mnt/MyDrive/contest/dacon/fake_news/3.Tokenizer/tokenizer_inputs.pickle', 'rb') as handle:
    tokenizer_inputs = pickle.load(handle)
# model
model = load_model('/content/mnt/MyDrive/contest/dacon/fake_news/5.Model/news_model')


Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RandomUniform in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Sub in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Mul in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op Add in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RandomUniform in device /job:

# 형태소 분석 + 전처리

In [7]:
# 임베딩 사이즈
EMBEDDING_DIM = 200
#불용어
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'] + ['흩', '희', '흰', '흴', '흽', '히', '힉', '힌', '힐', '힘', '힙', '힛', '힝', 'ㅏ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅛ', 'ㅜ', 'ㅠ', 'ㅡ', 'ㅣ', '', '은', '는', '이'] + ['섰', '성', '섶', '세', '섹', '센', '셀', '셈', '셉', '셋', '셌', '셍', '셑', '셔', '션', '셜', '셤', '셨', '셩', '셰', '셴', '셸', '소', '속', '솎', '손', '솔', '솜', '솝', '솟', '송', '솥', '솨', '솩', '쇄', '쇠', '쇤', '쇨', '쇳', '쇼', '쇽', '숀', '숄', '숍', '숏', '숑', '숖', '수', '숙', '순', '숟', '술', '숨', '숩', '숫', '숭', '숯', '숱', '숲', '숴', '쉐', '쉔', '쉘', '쉬', '쉭', '쉰', '쉴', '쉼', '쉽', '쉿', '슁', '슈', '슐', '슘', '슛', '슝', '스', '슥', '슨', '슬', '슭', '슴', '습', '슷', '승', '싀', '시', '식', '신', '싣', '실', '싫', '심', '십', '싯', '싱', '싶', '싸', '싹', '싼', '싿', '쌀', '쌈', '쌉', '쌌', '쌍', '쌓', '쌔', '쌕', '쌘', '쌤', '쌨', '쌩', '쌰', '썅', '써', '썩', '썬', '썰', '썸', '썹', '썼', '썽', '쎄', '쎈', '쎔', '쏘', '쏙', '쏜', '쏟', '쏠', '쏭', '쏴', '쏵', '쐋', '쐐', '쐬', '쐰', '쐴', '쑤', '쑥', '쑨', '쑵', '쑹', '쒀', '쒔', '쓰', '쓱', '쓴', '쓸', '씀', '씁', '씌', '씨', '씩', '씫', '씬', '씰', '씸', '씹', '씻', '씼', '씽', 'ㅇ', '아', '악', '안', '앉', '않', '알', '앍', '앎', '앏', '앓', '암', '압', '앗', '았', '앙', '앝', '앞', '애', '액', '앤', '앨', '앰', '앱', '앳', '앴', '앵', '야', '약', '얀', '얄', '얇', '얌', '얍', '얏', '얐', '양', '얕', '얗', '얘', '얜', '어', '억', '언', '얹', '얻', '얼', '얽', '엄', '업', '없', '엇', '었', '엉', '엊', '엌', '엎', '에', '엑', '엔', '엘', '엠', '엡', '엣', '엥', '여', '역', '엮', '연', '열', '엷', '염', '엽', '엾', '엿', '였', '영', '옅', '옆', '옇', '예', '옌', '옐', '옘', '옛', '오', '옥', '온', '올', '옭', '옮', '옯', '옰', '옳', '옴', '옵', '옷', '옹', '옻', '와', '왁', '완', '왈', '왎', '왑', '왓', '왔', '왕', '왜', '왝', '왠', '왱', '외', '왼', '욀', '요', '욕', '욘', '욜', '욤', '욥', '욧', '용', '우', '욱', '운', '울', '움', '웁', '웂', '웃', '웅', '워', '웍', '원', '월', '웜', '웟', '웠', '웨', '웩', '웬', '웰', '웹', '웽', '위', '윅', '윈', '윌', '윔', '윗', '윙', '유', '육', '윤', '율', '윰', '윱', '윳', '융', '윷', '으', '윽', '은', '읃', '을', '읊', '음', '읍', '읎', '읏', '응', '읒', '읓', '읔', '읕', '읖', '읗', '의', '읜', '읠', '이', '익', '인', '일', '읽', '잃', '임', '입', '잇', '있', '잉', '잊', '잌', '잍', '잎', 'ㅈ', '자', '작', '잔', '잖', '잘', '잠', '잡', '잣', '잤', '장', '잦', '재', '잭', '잰', '잴', '잼', '잽', '잿', '쟀', '쟁', '쟈', '쟌', '쟝', '쟤', '쟨', '저', '적', '전', '절', '젊', '젋', '점', '접', '젓', '젔', '정', '젖', '제', '젝', '젠', '젤', '젬', '젭', '젯', '져', '젼', '졌', '조', '족', '존', '졸', '좀', '좁', '종', '좆', '좇', '좋', '좌', '좔', '좦', '죄', '죈', '죌', '죔', '죠', '죤', '죵', '주', '죽', '준', '줄', '줌', '줍', '줏', '중', '줘', '줬', '줴', '쥐', '쥑', '쥔', '쥘', '쥠', '쥬', '쥴', '쥼', '즈', '즉', '즌', '즐', '즘', '즙', '증', '지', '직', '진', '짇', '질', '짊', '짐', '집', '짓', '징', '짖', '짙', '짚', '짜', '짝', '짠', '짢', '짤', '짧', '짬', '짭', '짯', '짰', '짱', '째', '짹', '짼', '쨈', '쨋', '쨌', '쨍', '쨔', '쨰', '쩌', '쩍', '쩐', '쩔', '쩜', '쩝', '쩡', '쩨', '쪄', '쪘', '쪼', '쪽', '쫀', '쫄', '쫌', '쫍', '쫑', '쫒', '쫓', '쫘', '쫙', '쬐', '쬔', '쬘', '쬠', '쭈', '쭉', '쭌', '쭐', '쭘', '쭙', '쭝', '쭤', '쮸', '쯔', '쯤', '쯧', '쯩', '쯽', '찌', '찍', '찐', '찔', '찜', '찝', '찟', '찡', '찢', '찦', '찧', 'ㅊ', '차', '착', '찬', '찮', '찰', '참', '찹', '찻', '찼', '창', '찾', '채', '책', '챈', '챌', '챔', '챕', '챗', '챘', '챙', '챠', '처', '척', '천', '철', '첨', '첩', '첫', '청', '체', '첸', '첼', '쳄', '쳅', '쳇', '쳉', '쳐', '쳔', '쳤', '초', '촉', '촌', '촐', '촘', '촛', '총', '촤', '촥', '촨', '촬', '촵', '최', '쵭', '쵸', '추', '축', '춘', '출', '춤', '춥', '춧', '충', '춰', '췄', '췌', '취', '췸', '츄', '츈', '츠', '측', '츨', '츰', '츱', '층', '치', '칙', '친', '칠', '칡', '침', '칩', '칫', '칭', 'ㅋ', '카', '칵', '칸', '칼', '캄', '캅', '캇', '캉', '캐', '캑', '캔', '캘', '캠', '캡', '캣', '캤', '캥', '캬', '캭', '커', '컥', '컨', '컫', '컬', '컴', '컵', '컷', '컸', '컹', '컽', '케', '켁', '켄', '켈', '켐', '켑', '켓', '켕', '켙', '켜', '켠', '켤', '켯', '켰', '켸', '코', '콕', '콘', '콜', '콤', '콥', '콧', '콩', '콰', '콱', '콸', '쾅', '쾌', '쾡', '쾨', '쾰', '쿄', '쿠', '쿡', '쿤', '쿨', '쿰', '쿵', '쿼', '퀀', '퀄', '퀘', '퀭', '퀴', '퀵', '퀸', '퀼', '큇', '큐', '큘', '크', '큭', '큰', '클', '큼', '큽', '킁', '키', '킥', '킨', '킬', '킴', '킵', '킷', '킹', 'ㅌ', '타', '탁', '탄', '탈', '탉', '탐', '탑', '탓', '탔', '탕', '태', '택', '탠', '탤', '탬', '탭', '탯', '탰', '탱', '탸', '턔', '터', '턱', '턴', '털', '텀', '텁', '텃', '텄', '텅', '테', '텍', '텐', '텔', '템', '텝', '텟', '텦', '텨', '텬', '텼', '톈', '토', '톡', '톤', '톨', '톰', '톱', '톳', '통', '퇘', '퇴', '툇', '툐', '투', '툭', '툰', '툴', '툼', '툽', '퉁', '퉈', '퉜', '퉤', '튀', '튁', '튄', '튈', '튑', '튕', '튜', '튤', '튬', '튱', '트', '특', '튼', '튿', '틀', '틈', '틉', '틋', '틔', '티', '틱', '틴', '틸', '팀', '팁', '팅', 'ㅍ', '파', '팍', '팎', '판', '팔', '팜', '팝', '팟', '팠', '팡', '팥', '패', '팩', '팬', '팰', '팸', '팻', '팼', '팽', '퍝', '퍼', '퍽', '펀', '펄', '펌', '펐', '펑', '페', '펙', '펜', '펠', '펨', '펩', '펫', '펭', '펴', '편', '펼', '폄', '폈', '평', '폐', '포', '폭', '폰', '폴', '폼', '폿', '퐁', '표', '푠', '푭', '푯', '푸', '푹', '푼', '풀', '풂', '품', '풋', '풍', '풔', '퓌', '퓨', '퓰', '프', '픈', '플', '픔', '픕', '피', '픽', '핀', '필', '핌', '핍', '핏', '핑', 'ㅎ', '하', '학', '한', '할', '핥', '함', '합', '핫', '항', '해', '핵', '핸', '핼', '햄', '햅', '햇', '했', '행', '햐', '향', '헀', '허', '헉', '헌', '헐', '험', '헙', '헛', '헝', '헤', '헥', '헨', '헬', '헴', '헵', '헷', '헸', '헹', '혀', '혁', '현', '혈', '혐', '협', '혓', '혔', '형', '혜', '혤', '호', '혹', '혼', '홀', '홈', '홉', '홋', '홍', '홑', '화', '확', '환', '활', '홥', '홧', '황', '홰', '홱', '횃', '횅', '회', '획', '횐', '횔', '횝', '횟', '횡', '효', '후', '훅', '훈', '훌', '훑', '훔', '훗', '훙', '훠', '훤', '훨', '훵', '훼', '휀', '휄', '휑', '휘', '휙', '휜', '휠', '휩', '휭', '휴', '휼', '흄', '흉', '흐', '흑', '흔', '흗', '흘', '흙', '흠', '흡', '흥'] + ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'ㄱ', '가', '각', '간', '갇', '갈', '갉', '감', '갑', '값', '갓', '갔', '강', '갖', '갗', '같', '갚', '갛', '개', '객', '갠', '갤', '갬', '갭', '갯', '갰', '갱', '갸', '갹', '갼', '걀', '걔', '걘', '걜', '거', '걱', '건', '걷', '걸', '검', '겁', '겂', '것', '겄', '겅', '겆', '겉', '겊', '겋', '게', '겐', '겔', '겜', '겝', '겟', '겠', '겡', '겨', '격', '겪', '견', '겯', '결', '겸', '겹', '겻', '겼', '경', '겿', '곁', '계', '곈', '곌', '곕', '곗', '고', '곡', '곤', '곧', '골', '곪', '곬', '곯', '곰', '곱', '곳', '공', '곶', '과', '곽', '관', '괄', '괌', '괍', '광', '괘', '괜', '괭', '괴', '괵', '괸', '괼', '굄', '굉', '교', '굔', '굘', '구', '국', '군', '굳', '굴', '굵', '굶', '굼', '굽', '굿', '궁', '궂', '궃', '궈', '권', '궐', '궜', '궤', '귀', '귄', '귈', '귐', '귑', '귓', '규', '균', '귤', '귭', '그', '극', '근', '귿', '글', '긁', '금', '급', '긋', '긍', '긔', '기', '긴', '긷', '길', '김', '깁', '깃', '깄', '깅', '깊', '까', '깍', '깎', '깐', '깔', '깜', '깝', '깟', '깠', '깡', '깥', '깨', '깬', '깰', '깸', '깻', '깼', '깽', '꺄', '꺠', '꺤', '꺼', '꺽', '꺾', '껀', '껄', '껌', '껍', '껏', '껐', '껑', '께', '껜', '껭', '껴', '껸', '꼈', '꼍', '꼐', '꼬', '꼭', '꼰', '꼴', '꼼', '꼽', '꼿', '꽁', '꽂', '꽃', '꽈', '꽉', '꽌', '꽝', '꽤', '꽥', '꽹', '꾀', '꾐', '꾸', '꾹', '꾼', '꿀', '꿇', '꿈', '꿉', '꿋', '꿍', '꿎', '꿏', '꿔', '꿨', '꿩', '꿰', '꿴', '꿸', '뀌', '뀐', '뀔', '뀜', '뀝', '끄', '끅', '끈', '끊', '끌', '끓', '끔', '끕', '끗', '끙', '끝', '끼', '끽', '낀', '낄', '낌', '낍', '낏', '낑', 'ㄴ', '나', '낙', '낚', '난', '낟', '날', '낡', '남', '납', '낫', '났', '낭', '낮', '낯', '낱', '낳', '내', '낵', '낸', '낼', '냄', '냅', '냇', '냈', '냉', '냐', '냑', '냔', '냘', '냠', '냥', '너', '넉', '넋', '넌', '널', '넑', '넓', '넘', '넙', '넛', '넜', '넝', '넣', '네', '넥', '넨', '넬', '넵', '넷', '넸', '넹', '녀', '녁', '년', '녈', '념', '녔', '녕', '녘', '녜', '노', '녹', '논', '놀', '놈', '놉', '놋', '농', '높', '놓', '놔', '놘', '놥', '놨', '뇌', '뇔', '뇨', '뇩', '뇰', '뇽', '누', '눅', '눈', '눌', '눔', '눕', '눗', '눙', '눠', '눴', '뉘', '뉜', '뉠', '뉨', '뉴', '뉼', '늄', '늉', '느', '늑', '는', '늗', '늘', '늙', '늠', '늡', '능', '늦', '늪', '늬', '니', '닉', '닌', '닐', '닒', '님', '닙', '닛', '닝', '닞', '닠', '닢', 'ㄷ', '다', '닥', '닦', '단', '닫', '달', '닭', '닮', '닳', '담', '답', '닷', '당', '닺', '닻', '닿', '대', '댁', '댄', '댈', '댐', '댑', '댓', '댔', '댕', '댜', '더', '덕', '던', '덜', '덞', '덟', '덤', '덥', '덧', '덩', '덫', '덮', '덯', '데', '덱', '덴', '델', '뎀', '뎁', '뎃', '뎅', '뎌', '뎐', '뎠', '뎡', '뎨', '뎬', '도', '독', '돈', '돋', '돌', '돐', '돔', '돕', '돗', '동', '돛', '돝', '돤', '돼', '됐', '되', '된', '될', '됨', '됩', '됫', '됬', '됴', '두', '둑', '둔', '둘', '둠', '둡', '둣', '둥', '둬', '뒀', '뒈', '뒤', '뒷', '뒹', '듀', '듈', '듐', '드', '득', '든', '듣', '들', '듦', '듬', '듭', '듯', '등', '듸', '듼', '디', '딕', '딘', '딛', '딜', '딤', '딥', '딧', '딨', '딩', '딪', '따', '딱', '딴', '딸', '땀', '땁', '땃', '땄', '땅', '땋', '때', '땐', '땔', '땜', '땟', '땠', '땡', '떄', '떠', '떡', '떤', '떨', '떫', '떱', '떳', '떴', '떵', '떻', '떼', '떽', '뗀', '뗄', '뗏', '뗐', '뗑', '또', '똑', '똔', '똘', '똥', '똬', '뙈', '뙤', '뚜', '뚝', '뚤', '뚫', '뚱', '뚸', '뛰', '뛴', '뛸', '뜀', '뜁', '뜨', '뜩', '뜬', '뜯', '뜰', '뜸', '뜹', '뜻', '띄', '띈', '띌', '띔', '띕', '띠', '띤', '띨', '띵', 'ㄹ', '라', '락', '란', '랄', '람', '랍', '랏', '랐', '랑', '랖', '랗', '래', '랙', '랜', '랠', '램', '랩', '랫', '랬', '랭', '랴', '략', '럇', '량', '러', '럭', '런', '럴', '럼', '럽', '럿', '렀', '렁', '렇', '레', '렉', '렌', '렐', '렘', '렙', '렛', '렜', '렝', '려', '력', '련', '렬', '렴', '렵', '렷', '렸', '령', '례', '롄', '롓', '로', '록', '론', '롤', '롬', '롭', '롯', '롱', '뢰', '룁', '료', '룐', '룡', '루', '룩', '룬', '룰', '룸', '룹', '룻', '룽', '뤄', '뤘', '뤼', '뤽', '륀', '류', '륙', '륜', '률', '륨', '륫', '륭', '르', '륵', '른', '를', '름', '릅', '릇', '릉', '릎', '릏', '리', '릭', '린', '릴', '림', '립', '릿', '링', 'ㅁ', '마', '막', '만', '많', '맏', '말', '맑', '맘', '맙', '맛', '맜', '망', '맞', '맡', '맣', '매', '맥', '맨', '맬', '맴', '맵', '맷', '맸', '맹', '맺', '맽', '먁', '머', '먹', '먼', '멀', '멈', '멉', '멋', '멍', '멎', '멓', '메', '멕', '멘', '멛', '멜', '멤', '멥', '멧', '멨', '멩', '멫', '며', '멱', '면', '멷', '멸', '몃', '몄', '명', '몇', '모', '목', '몫', '몬', '몰', '몸', '몹', '못', '몽', '뫼', '묏', '묘', '무', '묵', '묶', '문', '묻', '물', '묽', '뭄', '뭅', '뭇', '뭉', '뭍', '뭏', '뭐', '뭔', '뭘', '뭡', '뭣', '뮈', '뮌', '뮐', '뮙', '뮤', '뮬', '뮴', '므', '믄', '믈', '믐', '믓', '미', '믹', '민', '믿', '밀', '밈', '밉', '밋', '밌', '밍', '및', '밑', 'ㅂ', '바', '박', '밖', '반', '받', '발', '밝', '밟', '밣', '밤', '밥', '밧', '방', '밭', '배', '백', '밴', '밸', '뱀', '뱁', '뱃', '뱄', '뱅', '뱉', '버', '벅', '번', '벋', '벌', '범', '법', '벗', '벙', '벚', '벝', '벟', '베', '벡', '벤', '벧', '벨', '벱', '벳', '벵', '벼', '벽', '변', '별', '볍', '볏', '볐', '병', '볕', '보', '복', '볶', '본', '볼', '봄', '봅', '봇', '봉', '봐', '봤', '봬', '뵀', '뵈', '뵌', '뵐', '뵘', '뵙', '뵜', '뵤', '부', '북', '분', '불', '붉', '붐', '붑', '붓', '붕', '붙', '뷔', '뷰', '뷴', '뷸', '븀', '브', '븍', '븐', '블', '비', '빅', '빈', '빌', '빔', '빕', '빗', '빙', '빚', '빛', '빠', '빡', '빤', '빨', '빰', '빳', '빴', '빵', '빻', '빼', '빽', '뺀', '뺄', '뺌', '뺏', '뺐', '뺑', '뺨', '뻐', '뻑', '뻔', '뻗', '뻘', '뻣', '뻤', '뻥', '뻬', '뼁', '뼈', '뼉', '뼘', '뼛', '뼜', '뽀', '뽄', '뽈', '뽐', '뽑', '뽕', '뾰', '뿅', '뿌', '뿍', '뿐', '뿔', '뿜', '쁘', '쁜', '쁠', '쁨', '쁩', '삐', '삑', '삔', '삘', '삣', '삥', 'ㅅ', '사', '삭', '삯', '산', '살', '삵', '삶', '삼', '삽', '삿', '샀', '상', '샅', '샆', '새', '색', '샌', '샐', '샘', '샙', '샛', '샜', '생', '샤', '샥', '샨', '샬', '샴', '샵', '샷', '샹', '섀', '서', '석', '섞', '선', '섣', '설', '섦', '섧', '섬', '섭', '섯']


def text_preprocessing(paragraph):
    '''전처리 프로세스'''
    
    tokenizer = Mecab()
    
    sentence = re.sub('([a-zA-Z])','',paragraph)
    sentence = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',sentence)
    sentence = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ·!』\\‘|\(\)\[\]\<\>`\'…》]','',sentence)
    token = tokenizer.nouns(sentence)

    token = [t for t in token if t not in stopwords or type(t) != float]

    
    return token


# 토큰화된 결과물을 저장할 리스트
token_ls = []
test_token_ls = test_df['content']


for i in test_token_ls:
    tmp = text_preprocessing(i)
    token_ls.append(tmp)



In [8]:
# 몇번째 단어인지, 단어를 숫자로 변환
input_sequences = tokenizer_inputs.texts_to_sequences(token_ls)
word2idx_inputs = tokenizer_inputs.word_index

MAIN_MAX_LEN_INPUT = 50
encoder_inputs = pad_sequences(input_sequences, maxlen = MAIN_MAX_LEN_INPUT)
num_words = 20000


embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, idx in word2idx_inputs.items():
    if idx < num_words:
        try:
            embedding_vector = word2vec[word]
            
            if embedding_vector is not None:
                embedding_matrix[idx-1, :] = word2vec[word]
        
        except:
            embedding_vector = 0
            embedding_matrix[idx-1, :] = 0

pre_data = pad_sequences(input_sequences, maxlen = MAIN_MAX_LEN_INPUT)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


# 예측

In [9]:
p = model.predict(pre_data)
result = [np.argmax(i) for i in np.array(p)]

sub = pd.read_csv("/content/mnt/MyDrive/contest/dacon/fake_news/1.Data/sample_submission.csv")
sub['info'] = result

Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Identity in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op RangeDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op MapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op PrefetchDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op FlatMapDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op TensorDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op RepeatDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ZipDataset in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ParallelMapDatasetV2 in device /job:localhost/replica:0/task:0/device:CPU:0
Executing op ReadVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Ide

In [10]:
print(time.time() - start)

193.83389401435852


In [11]:
sub.to_csv("/content/submission.csv")